# A basic template to apply any models to train on the collected data

In [7]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import zscore
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
import graphviz
plt.rcParams['figure.figsize'] = [18, 10]

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split



In [8]:
trainDF = pd.read_csv('../outputs/trainXY.csv',parse_dates=['date_implement'])
trainDF = trainDF.sample(frac=1)

In [9]:
trainDF.head()
trainDF = trainDF.dropna()

In [10]:
# Finalizing Indexes so we can drop state names 
trainDF = trainDF.reset_index(drop=True)

# Get a list of columns with features 
columns = trainDF.columns.to_list()

In [11]:
Y_columns = [x for x in columns if 'Y_' in x]

In [12]:
X_columns = [x for x in columns if 'Y_' not in x and 'Unnamed' not in x]

X_columns.remove('date_implement')
# X_columns.remove('policy')
X_columns.remove('state_x')
X_columns.remove('policy_type')
X_columns.remove('stateName')
X_columns.remove('CEN_stateCode')
X_columns.remove('submission_date')
X_columns.remove('state_y')

X_columns.remove('caseInterpolate_gauss3')
X_columns.remove('caseInterpolate_MA7')
X_columns.remove('caseInterpolate_savitzky31_3')
X_columns.remove('caseInterpolate_gauss8')
X_columns.remove('new_case_zscore')
X_columns.remove('new_case')



In [13]:
X = trainDF[X_columns]
y = trainDF[Y_columns]


In [14]:
y

,Y_TREND_caseInterpolate_MA7_14,Y_TRENDQUANT_caseInterpolate_MA7_14,Y_SLOPE_caseInterpolate_MA7_14,Y_TREND_caseInterpolate_MA7_21,Y_TRENDQUANT_caseInterpolate_MA7_21,Y_SLOPE_caseInterpolate_MA7_21,Y_TREND_caseInterpolate_MA7_28,Y_TRENDQUANT_caseInterpolate_MA7_28,Y_SLOPE_caseInterpolate_MA7_28,Y_TREND_caseInterpolate_gauss8_14,Y_TRENDQUANT_caseInterpolate_gauss8_14,Y_SLOPE_caseInterpolate_gauss8_14,Y_TREND_caseInterpolate_gauss8_21,Y_TRENDQUANT_caseInterpolate_gauss8_21,Y_SLOPE_caseInterpolate_gauss8_21,Y_TREND_caseInterpolate_gauss8_28,Y_TRENDQUANT_caseInterpolate_gauss8_28,Y_SLOPE_caseInterpolate_gauss8_28,Y_TREND_caseInterpolate_gauss3_14,Y_TRENDQUANT_caseInterpolate_gauss3_14,Y_SLOPE_caseInterpolate_gauss3_14,Y_TREND_caseInterpolate_gauss3_21,Y_TRENDQUANT_caseInterpolate_gauss3_21,Y_SLOPE_caseInterpolate_gauss3_21,Y_TREND_caseInterpolate_gauss3_28,Y_TRENDQUANT_caseInterpolate_gauss3_28,Y_SLOPE_caseInterpolate_gauss3_28
0,-1.0,-4.224490,-2.754615,-1.0,-2.442177,-2.754615,-1.0,-2.525510,-2.754615,-1.0,-1.785714,-2.142308,-1.0,-2.000000,-2.142308,-1.0,-2.178571,-2.142308,-1.0,-1.285714,-2.693846,-1.0,-2.095238,-2.693846,-1.0,-1.321429,-2.693846
1,-1.0,-89.561224,-154.239451,-1.0,-148.353741,-154.239451,-1.0,-105.852041,-154.239451,-1.0,-93.642857,-113.010769,-1.0,-107.380952,-113.010769,-1.0,-111.321429,-113.010769,-1.0,-80.000000,-126.642308,-1.0,-120.047619,-126.642308,-1.0,-114.142857,-126.642308
2,-1.0,-13.204082,-47.519341,-1.0,-43.435374,-47.519341,-1.0,-25.408163,-47.519341,-1.0,-16.857143,-24.621538,-1.0,-22.523810,-24.621538,-1.0,-18.714286,-24.621538,-1.0,-8.857143,-30.713077,-1.0,-25.571429,-30.713077,-1.0,-36.178571,-30.713077
3,1.0,0.275510,0.099780,1.0,0.183673,0.099780,1.0,0.137755,0.099780,1.0,5.428571,2.532308,1.0,3.714286,2.532308,1.0,2.785714,2.532308,1.0,1.000000,0.316923,1.0,0.666667,0.316923,1.0,0.500000,0.316923
4,-1.0,-99.306122,-97.343297,-1.0,-107.741497,-97.343297,-1.0,-61.984694,-97.343297,-1.0,-96.285714,-85.182308,-1.0,-87.571429,-85.182308,-1.0,-69.928571,-85.182308,-1.0,-101.285714,-104.553077,-1.0,-110.142857,-104.553077,-1.0,-76.285714,-104.553077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,-1.0,-0.173469,0.269890,1.0,0.013605,0.269890,-1.0,-0.066327,0.269890,0.0,0.000000,-0.065385,-1.0,-0.047619,-0.065385,-1.0,-0.250000,-0.065385,-1.0,-0.357143,0.093077,-1.0,-0.095238,0.093077,-1.0,-0.142857,0.093077
1579,1.0,1.316327,0.500220,1.0,0.891156,0.500220,1.0,0.668367,0.500220,1.0,11.214286,5.700000,1.0,7.809524,5.700000,1.0,5.892857,5.700000,1.0,4.285714,1.653846,1.0,2.904762,1.653846,1.0,2.178571,1.653846
1580,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1581,1.0,82.959184,104.718242,1.0,95.782313,104.718242,1.0,99.928571,104.718242,1.0,71.857143,86.565385,1.0,82.095238,86.565385,1.0,87.035714,86.565385,1.0,71.428571,90.426923,1.0,80.904762,90.426923,1.0,93.714286,90.426923


# Encoding Policy Names 

In [15]:
X['policy'].unique()

array(['C6_Stay at home requirements', 'C4_Restrictions on gatherings',
       'C2_Workplace closing', 'H7_Vaccination policy',
       'C1_School closing', 'C7_Restrictions on internal movement',
       'H6_Facial Coverings', 'C8_International travel controls',
       'C3_Cancel public events', 'H3_Contact tracing',
       'H1_Public information campaigns', 'H2_Testing policy',
       'C5_Close public transport'], dtype=object)

In [16]:
le = preprocessing.LabelEncoder()
le.fit(X['policy'])
print(le.classes_)

['C1_School closing' 'C2_Workplace closing' 'C3_Cancel public events'
 'C4_Restrictions on gatherings' 'C5_Close public transport'
 'C6_Stay at home requirements' 'C7_Restrictions on internal movement'
 'C8_International travel controls' 'H1_Public information campaigns'
 'H2_Testing policy' 'H3_Contact tracing' 'H6_Facial Coverings'
 'H7_Vaccination policy']


In [17]:
X['policy'] = pd.Series(le.transform(X['policy']))
original_policies_decoded = pd.Series(le.inverse_transform(X['policy']))

/var/folders/66/w1588p9n1654b1yky4q5nz580000gn/T/ipykernel_5777/2867200982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['policy'] = pd.Series(le.transform(X['policy']))


------

-------

# Train Models Here

In [18]:
y

,Y_TREND_caseInterpolate_MA7_14,Y_TRENDQUANT_caseInterpolate_MA7_14,Y_SLOPE_caseInterpolate_MA7_14,Y_TREND_caseInterpolate_MA7_21,Y_TRENDQUANT_caseInterpolate_MA7_21,Y_SLOPE_caseInterpolate_MA7_21,Y_TREND_caseInterpolate_MA7_28,Y_TRENDQUANT_caseInterpolate_MA7_28,Y_SLOPE_caseInterpolate_MA7_28,Y_TREND_caseInterpolate_gauss8_14,Y_TRENDQUANT_caseInterpolate_gauss8_14,Y_SLOPE_caseInterpolate_gauss8_14,Y_TREND_caseInterpolate_gauss8_21,Y_TRENDQUANT_caseInterpolate_gauss8_21,Y_SLOPE_caseInterpolate_gauss8_21,Y_TREND_caseInterpolate_gauss8_28,Y_TRENDQUANT_caseInterpolate_gauss8_28,Y_SLOPE_caseInterpolate_gauss8_28,Y_TREND_caseInterpolate_gauss3_14,Y_TRENDQUANT_caseInterpolate_gauss3_14,Y_SLOPE_caseInterpolate_gauss3_14,Y_TREND_caseInterpolate_gauss3_21,Y_TRENDQUANT_caseInterpolate_gauss3_21,Y_SLOPE_caseInterpolate_gauss3_21,Y_TREND_caseInterpolate_gauss3_28,Y_TRENDQUANT_caseInterpolate_gauss3_28,Y_SLOPE_caseInterpolate_gauss3_28
0,-1.0,-4.224490,-2.754615,-1.0,-2.442177,-2.754615,-1.0,-2.525510,-2.754615,-1.0,-1.785714,-2.142308,-1.0,-2.000000,-2.142308,-1.0,-2.178571,-2.142308,-1.0,-1.285714,-2.693846,-1.0,-2.095238,-2.693846,-1.0,-1.321429,-2.693846
1,-1.0,-89.561224,-154.239451,-1.0,-148.353741,-154.239451,-1.0,-105.852041,-154.239451,-1.0,-93.642857,-113.010769,-1.0,-107.380952,-113.010769,-1.0,-111.321429,-113.010769,-1.0,-80.000000,-126.642308,-1.0,-120.047619,-126.642308,-1.0,-114.142857,-126.642308
2,-1.0,-13.204082,-47.519341,-1.0,-43.435374,-47.519341,-1.0,-25.408163,-47.519341,-1.0,-16.857143,-24.621538,-1.0,-22.523810,-24.621538,-1.0,-18.714286,-24.621538,-1.0,-8.857143,-30.713077,-1.0,-25.571429,-30.713077,-1.0,-36.178571,-30.713077
3,1.0,0.275510,0.099780,1.0,0.183673,0.099780,1.0,0.137755,0.099780,1.0,5.428571,2.532308,1.0,3.714286,2.532308,1.0,2.785714,2.532308,1.0,1.000000,0.316923,1.0,0.666667,0.316923,1.0,0.500000,0.316923
4,-1.0,-99.306122,-97.343297,-1.0,-107.741497,-97.343297,-1.0,-61.984694,-97.343297,-1.0,-96.285714,-85.182308,-1.0,-87.571429,-85.182308,-1.0,-69.928571,-85.182308,-1.0,-101.285714,-104.553077,-1.0,-110.142857,-104.553077,-1.0,-76.285714,-104.553077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,-1.0,-0.173469,0.269890,1.0,0.013605,0.269890,-1.0,-0.066327,0.269890,0.0,0.000000,-0.065385,-1.0,-0.047619,-0.065385,-1.0,-0.250000,-0.065385,-1.0,-0.357143,0.093077,-1.0,-0.095238,0.093077,-1.0,-0.142857,0.093077
1579,1.0,1.316327,0.500220,1.0,0.891156,0.500220,1.0,0.668367,0.500220,1.0,11.214286,5.700000,1.0,7.809524,5.700000,1.0,5.892857,5.700000,1.0,4.285714,1.653846,1.0,2.904762,1.653846,1.0,2.178571,1.653846
1580,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1581,1.0,82.959184,104.718242,1.0,95.782313,104.718242,1.0,99.928571,104.718242,1.0,71.857143,86.565385,1.0,82.095238,86.565385,1.0,87.035714,86.565385,1.0,71.428571,90.426923,1.0,80.904762,90.426923,1.0,93.714286,90.426923


In [19]:
X = X.apply(zscore)
y = y.apply(zscore)

In [20]:
from sklearn.datasets import load_iris
from sklearn import tree

In [21]:
clf = tree.DecisionTreeRegressor()

In [22]:
y_selected = y[['Y_TRENDQUANT_caseInterpolate_MA7_14']]

In [23]:
clf = clf.fit(X, y_selected)

In [24]:
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris") 

'iris.pdf'

In [25]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                     feature_names=X.columns.to_list(), 
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.render('op')

'op.pdf'

In [92]:
!open .

In [96]:
# Random Forest 

regr = RandomForestRegressor(max_depth=100, random_state=0)
regr.fit(X, y_selected)

/var/folders/66/w1588p9n1654b1yky4q5nz580000gn/T/ipykernel_37377/3596666854.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X, y_selected)


RandomForestRegressor(max_depth=100, random_state=0)

In [97]:
def imp_df(column_names, importances):
    data = {
        'Feature': column_names,
        'Importance': importances,
    }
    df = pd.DataFrame(data) \
        .set_index('Feature') \
        .sort_values('Importance', ascending=False)

    return df

In [98]:
base_imp = imp_df(X.columns, regr.feature_importances_)
print(base_imp)

                                                 Importance
Feature                                                    
policy                                             0.469987
metric_change                                      0.187818
FD_moreThan10kCitizens                             0.021709
S1_people_fully_vaccinated_per_hundred             0.019639
S1_people_vaccinated                               0.018727
S1_people_fully_vaccinated                         0.018476
CEN_Without Health Care Coverage                   0.017234
CEN_Hispanic or Latino (of any race)               0.016954
S2_people_fully_vaccinated                         0.011627
CEN_Total Employer Establishments                  0.011586
CEN_Total Population                               0.010546
S2_people_vaccinated                               0.010059
S4_people_fully_vaccinated                         0.009749
CEN_Total Housing Units                            0.009589
FD_lessThan10kCitizens                  

In [26]:
from sklearn.kernel_ridge import KernelRidge
import numpy as np
n_samples, n_features = 10, 5
rng = np.random.RandomState(0)

krr = KernelRidge(alpha=1.0)
krr.fit(X, y_selected)

KernelRidge(alpha=1.0)

In [27]:
krr.predict(X.loc[])

KernelRidge(alpha=1.0)